<a href="https://colab.research.google.com/github/Yyzhang2000/AI-Cookbook/blob/main/llm_rag_multimodal_%E2%9C%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdf2image Pillow chromadb torch torchvision torchaudio transformers librosa ipython open-clip-torch qwen_vl_utils
!pip install -U transformers

In [2]:
!apt-get update
!apt-get install -y poppler-utils

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,642 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,907 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [3]:
import os
from pdf2image import convert_from_path
from PIL import Image
import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction

import torch
from transformers import CLIPProcessor, CLIPModel, WhisperProcessor, WhisperForConditionalGeneration, Qwen2VLForConditionalGeneration, Qwen2VLProcessor

import librosa
from sentence_transformers import SentenceTransformer
from qwen_vl_utils import process_vision_info
from IPython.display import display, Image as IPImage

In [4]:
OUTPUT_DIR = "dataset"
IMAGE_OUTPUT_DIR = "extracted_images"
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [9]:
IMAGE_EMBEDDING_MODEL = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
IMAGE_PROCESSOR = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

AUDIO_PROCESSOR = WhisperProcessor.from_pretrained("openai/whisper-small")
AUDIO_EMBEDDING_MODEL = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [7]:
def convert_pdfs_to_images(
        folder,
        image_output_dir
):
    if not os.path.exists(image_output_dir):
        os.makedirs(image_output_dir)

    pdf_files = [f for f in os.listdir(folder) if f.endswith('.pdf')]
    all_images = {}

    for doc_id, pdf_file in enumerate(pdf_files):
        pdf_path = os.path.join(folder, pdf_file)
        images = convert_from_path(pdf_path, dpi=100)

        image_paths = []
        for i, image in enumerate(images):
            image_path = os.path.join(image_output_dir, f"{doc_id}_page_{i}.png")

            image.save(image_path, 'PNG')
            image_paths.append(image_path)

        all_images[doc_id] = image_paths

    return all_images

def embed_images(image_paths):
    embeddings = []
    for path in image_paths:
        image = Image.open(path)

        inputs = IMAGE_PROCESSOR(images = image, return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            image_embedding = IMAGE_EMBEDDING_MODEL.get_image_features(
                **inputs
            ).cpu().numpy()

        embeddings.append(image_embedding)

    return embeddings


In [ ]:
def transcribe_audio(
        audio_path,
        chunk_length = 30
):
    audio, sr = librosa.load(audio_path, sr=16000)
    chunk_size = chunk_length * sr
    chunks = [audio[i:i + chunk_size] for i in range(0, len(audio), chunk_size)]

    transcription_chunks = []
    for chunk in chunks:
        inputs = AUDIO_PROCESSOR(chunk, sampling_rate=sr, return_tensors="pt").to(device)
        inputs["attention_mask"] = torch.ones_like(inputs.input_features)
        with torch.no_grad():
            predicted_ids = AUDIO_EMBEDDING_MODEL.generate(**inputs, max_length=448)
        chunk_transcription = AUDIO_EMBEDDING_MODEL.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        transcription_chunks.append(chunk_transcription)

    full_transcription = " ".join(transcription_chunks)
    return full_transcription, transcription_chunks

In [ ]:
all_images = convert_pdfs_to_images(OUTPUT_DIR, IMAGE_OUTPUT_DIR)
image_embeddings = {}
for doc_id, paths in all_images.items():
    image_embeddings[doc_id] = embed_images(paths)


audio_files = [f for f in os.listdir(output_dir) if f.endswith('.mp3')]
audio_transcriptions = {}
for audio_id, audio_file in enumerate(audio_files):
    audio_path = os.path.join(output_dir, audio_file)
    full_transcription, transcription_chunks = transcribe_audio(audio_path)
    audio_transcriptions[audio_id] = {
        "full_transcription": full_transcription,
        "chunks": transcription_chunks
    }

In [ ]:
client = chromadb.PersistentClient(path="chroma_db")
embedding_function = OpenCLIPEmbeddingFunction()
text_embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [10]:
try:
    client.delete_collection(name="image_collection")
    client.delete_collection(name="audio_collection")
    print("Deleted existing collections.")
except Exception as e:
    print(f"Collections do not exist or could not be deleted: {e}")


Collections do not exist or could not be deleted: name 'client' is not defined


In [ ]:

image_collection = client.create_collection(name="image_collection", embedding_function=embedding_function)
audio_collection = client.create_collection(name="audio_collection")

for doc_id, embeddings in image_embeddings.items():
    for i, embedding in enumerate(embeddings):
        image_collection.add(
            ids=[f"image_{doc_id}_{i}"],
            embeddings=[embedding.flatten().tolist()],
            metadatas=[{"doc_id": str(doc_id), "image_path": all_images[doc_id][i]}]
        )

for audio_id, transcription_data in audio_transcriptions.items():
    transcription_chunks = transcription_data["chunks"]
    for chunk_id, chunk in enumerate(transcription_chunks):
        chunk_embedding = text_embedding_model.encode(chunk)
        audio_collection.add(
            ids=[f"audio_{audio_id}_chunk_{chunk_id}"],
            embeddings=[chunk_embedding.tolist()],
            metadatas=[{
                "audio_id": str(audio_id),
                "audio_path": audio_files[audio_id],
                "chunk_id": str(chunk_id)
            }],
            documents=[chunk]
        )

In [ ]:
def retrieve_data(query, top_k=2):

    query_embedding_image = embedding_function([query])[0]  # OpenCLIP for image collection
    query_embedding_audio = text_embedding_model.encode(query)  # SentenceTransformer for audio collection

    image_results = image_collection.query(
        query_embeddings=[query_embedding_image],
        n_results=top_k
    )

    audio_results = audio_collection.query(
        query_embeddings=[query_embedding_audio.tolist()],
        n_results=top_k
    )

    retrieved_images = [metadata["image_path"] for metadata in image_results["metadatas"][0] if "image_path" in metadata]
    retrieved_chunks = audio_results["documents"][0] if "documents" in audio_results else []

    return retrieved_images, retrieved_chunks

query = "What are the healthiest ingredients to use in recipe you have?"
retrieved_images, retrieved_chunks = retrieve_data(query)
print("Retrieved Images:", retrieved_images)
print("Retrieved Audio Chunks:", retrieved_chunks)

In [ ]:
vl_model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
).cuda().eval()

min_pixels = 256 * 256
max_pixels = 1024 * 1024
vl_model_processor = Qwen2VLProcessor.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    min_pixels=min_pixels,
    max_pixels=max_pixels
)

In [ ]:
chat_template = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": retrieved_images[0]},  # First retrieved image
            {"type": "image", "image": retrieved_images[1]},  # Second retrieved image
            {"type": "text", "text": query},  # User query
            {"type": "text", "text": "Audio Context: " + " ".join(retrieved_chunks)}  # Include audio data
        ],
    }
]

text = vl_model_processor.apply_chat_template(
    chat_template, tokenize=False, add_generation_prompt=True
)

image_inputs, _ = process_vision_info(chat_template)
inputs = vl_model_processor(
    text=[text],
    images=image_inputs,
    padding=True,
    return_tensors="pt",
).to("cuda")

generated_ids = vl_model.generate(**inputs, max_new_tokens=100)
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = vl_model_processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print(output_text[0])

In [ ]:
from PIL import Image
from IPython.display import display

# Function to display retrieved images
def display_retrieved_images(retrieved_images):
    for image_path in retrieved_images:
        try:
            # Open the image using PIL
            img = Image.open(image_path)
            print(f"Displaying image: {image_path}")
            display(img)  # Display the image in a notebook or GUI
        except Exception as e:
            print(f"Error displaying image {image_path}: {e}")


In [ ]:
display_retrieved_images(retrieved_images)